# Prédiction with XGBoost et Scikit-learn : Classification

### Récupération des fichiers csv with pandas

In [1]:
from pandas import read_csv
train_full = read_csv('csv_file/train_full.csv', index_col=0)
test_kaggle = read_csv('csv_file/dataset_testKaggle.csv', index_col=0)

print(train_full.shape)
print(test_kaggle.shape)

(150000, 19)
(101503, 19)


In [2]:
import ipynb.fs.defs.GiveMeCredit_SubmissionKaggle_BigML as gmc_sk

train_prepared = gmc_sk.change_dataset(train_full)
test_prepared = gmc_sk.change_dataset(test_kaggle)

print(train_prepared.shape)
print(test_prepared.shape)

(150000, 19)
(101503, 19)


### Définition des X et y pour l'ensemble train et X pour test

In [3]:
target_column = 'SeriousDlqin2yrs'
y_train = train_prepared[target_column].values
print(y_train.shape)

(150000,)


In [4]:
from pprint import pprint
X_train = train_prepared.drop(target_column, axis=1).values
pprint(X_train.shape)

(150000, 18)


In [5]:
X_test = test_prepared.drop(target_column, axis=1).values
print(X_test.shape)

(101503, 18)


## Machine Learning
### XGBoost

##### Model

In [6]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier()

##### Training

In [7]:
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

##### Prediction

In [14]:
y_test_xgb = model_xgb.predict_proba(X_test)
y_test_xgb

array([[0.9309477 , 0.06905226],
       [0.9568914 , 0.04310859],
       [0.9849723 , 0.01502773],
       ...,
       [0.99400246, 0.00599755],
       [0.9151748 , 0.0848252 ],
       [0.9578275 , 0.0421725 ]], dtype=float32)

In [9]:
y_test_xgb.shape

(101503, 2)

Prédictions 1probability

In [15]:
y_test_xgb_1p = y_test_xgb[:,1]
y_test_xgb_1p

array([0.06905226, 0.04310859, 0.01502773, ..., 0.00599755, 0.0848252 ,
       0.0421725 ], dtype=float32)

##### Submission Kaggle

In [16]:
import numpy as np
test_Id = np.arange(1, 101504)
test_Id

array([     1,      2,      3, ..., 101501, 101502, 101503])

In [17]:
import kaggle
import pandas as pd

d_xgb ={'Id':test_Id, 'Probability':y_test_xgb_1p}

sub_xgb = pd.DataFrame(data=d_xgb)
sub_xgb.shape

(101503, 2)

In [18]:
sub_xgb.describe()

,Id,Probability
count,101503.000000,101503.000000
mean,50752.000000,0.066956
std,29301.536524,0.114145
min,1.000000,0.004716
25%,25376.500000,0.011038
50%,50752.000000,0.021322
75%,76127.500000,0.066900
max,101503.000000,0.946606


In [19]:
sub_xgb.to_csv('csv_file/prediction_GMSC_XGB.csv', index=False)

In [20]:
submission_file = 'csv_file/prediction_GMSC_XGB.csv'
kaggle.api.competition_submit(submission_file, "XGBoost_Classifier", "GiveMeSomeCredit")

100%|██████████| 1.70M/1.70M [00:05<00:00, 325kB/s] 


Successfully submitted to Give Me Some Credit

### Scikit-Learn

##### Model

In [21]:
from sklearn.ensemble import RandomForestClassifier
model_skl = RandomForestClassifier()

##### Training

In [22]:
model_skl.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

##### Prediction

In [25]:
y_test_skl = model_skl.predict_proba(X_test)
y_test_skl

array([[1. , 0. ],
       [0.9, 0.1],
       [0.9, 0.1],
       ...,
       [1. , 0. ],
       [0.9, 0.1],
       [1. , 0. ]])

Prédictions 1probability

In [24]:
y_test_skl_1p = y_test_skl[:,1]
y_test_skl_1p

array([0. , 0.1, 0.1, ..., 0. , 0.1, 0. ])

##### Submission Kaggle

In [26]:
import kaggle
import pandas as pd

sub_skl = pd.DataFrame()

sub_skl['Id'] = test_Id
sub_skl['Probability'] = y_test_skl_1p

sub_skl.to_csv('csv_file/prediction_GMSC_SKl.csv', index=False)

In [27]:
submission_file = 'csv_file/prediction_GMSC_Skl.csv'
kaggle.api.competition_submit(submission_file, "Sklearn_RandomClassifier", "GiveMeSomeCredit")

100%|██████████| 984k/984k [00:02<00:00, 351kB/s]  


Successfully submitted to Give Me Some Credit